In [43]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
import pandas as pd
import re
from threading import Thread
import collections
import json
from datetime import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pickle

In [44]:
filename = '../machine learning models/rating_predicition_from_player_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [45]:
df = pd.read_csv('../data/processed.csv')

In [46]:
df.head()

,Name,Overall Rating,Potential,Value,Wage,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,...,GK Positioning,GK Reflexes,Positions,Nationality,Age,Height,Weight,Primary Position,Club,National Team
0,Matty Cash,78.0,83.0,20000000.0,50000.0,78.0,66.0,49.0,70.0,48.0,...,13.0,8.0,"['RB', 'RWB']",Poland,23.0,179.0,74.0,RB,Aston Villa,Poland
1,Gerónimo Rulli,81.0,81.0,17500000.0,33000.0,12.0,15.0,12.0,36.0,16.0,...,79.0,82.0,['GK'],Argentina,29.0,189.0,84.0,GK,Villarreal CF,No national team
2,Donyell Malen,79.0,85.0,28500000.0,56000.0,71.0,79.0,69.0,75.0,74.0,...,10.0,14.0,['ST'],Netherlands,22.0,179.0,78.0,ST,Borussia Dortmund,Netherlands
3,Oleksandr Zinchenko,79.0,82.0,22000000.0,90000.0,81.0,58.0,70.0,81.0,61.0,...,11.0,11.0,['LB'],Ukraine,24.0,175.0,64.0,LB,Manchester City,Ukraine
4,Rodrigo Hernández Cascante,87.0,89.0,90000000.0,190000.0,63.0,64.0,78.0,86.0,59.0,...,14.0,8.0,['CDM'],Spain,25.0,191.0,82.0,CDM,Manchester City,Spain


In [47]:
inp = []
countries = []
for country,temp_df in df.groupby('Nationality'):
    
    a = list(temp_df['Overall Rating'])
    a.sort(reverse=True)
    if len(a) >= 11:
        inp.append((country,a[0:11]))
        countries.append(country)



In [48]:
links = []
r = requests.get('https://sofifa.com/teams?type=national')
soup = bs(r.content)

page = soup.select('td > a')
a = 0
for i in page:
    if a % 2 == 0:
        links.append('https://sofifa.com'+i['href'])
    a += 1

In [49]:
whatever = []

for link in links:
    soup = bs(requests.get(link).content)
    name = soup.find('h1').get_text()
    card = soup.find('section',attrs={'class':'card'})
    div = card.find('div').get_text()
    pattern = re.compile(r'\d+')
    rating = int(pattern.findall(div)[0])
    whatever.append((name,rating))
    
      

In [50]:
whatever = dict(whatever)

In [51]:
input = []
exact = []
for i in inp:
    if i[0] in whatever:
        input.append(i[1])
        exact.append(whatever[i[0]])

In [52]:
len(input),len(exact)

(33, 33)

In [53]:
predictions = loaded_model.predict(input)

In [54]:
r2_score(exact, predictions)

0.9472691003766895